In [18]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
from PIL import Image, ImageTk
import pandas as pd
import csv
import os

# Load CSV
def load_csv():
    filepath = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
    if filepath:
        global df, current_index
        df = pd.read_csv(filepath)
        current_index = 0
        show_content()

# Show content
def show_content():
    if current_index >= len(df):
        messagebox.showinfo("End", "You've reached the end of the list.")
        return

    row = df.iloc[current_index]
    source = row["Fuente"]
    filepath = row["filename"]
    text = row["text"]

    for widget in content_frame.winfo_children():
        widget.destroy()

    # Display text or image
    if source == "Imágenes/Internet" and os.path.exists(filepath):
        img = Image.open(filepath)
        img = img.resize((400, 300), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(img)
        label = tk.Label(content_frame, image=img)
        label.image = img  # keep a reference to avoid garbage collection
        label.pack()
    else:
        label = tk.Label(content_frame, text=text, wraplength=400)
        label.pack()

# Save results
def save_result():
    row = df.iloc[current_index]
    rating = funny_slider.get()
    funny = funny_var.get()
    not_funny = not_funny_var.get()
    
    with open("results.csv", "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([row["id"], row["text"], row["source"], row["filepath"], funny, not_funny, rating])

# Move to the next item
def next_item():
    save_result()
    global current_index
    current_index += 1
    show_content()

# Keyboard shortcuts
def funny_shortcut(event=None):
    funny_var.set(True)
    not_funny_var.set(False)

def not_funny_shortcut(event=None):
    funny_var.set(False)
    not_funny_var.set(True)

# Slider control with numpad keys
def set_slider_value(value):
    funny_slider.set(value)

# Setup tkinter
root = tk.Tk()
root.title("Joke Rating App")
root.geometry("500x600")
root.configure(bg="#f5f5f5")

# Load CSV button
load_btn = ttk.Button(root, text="Load CSV", command=load_csv)
load_btn.pack(pady=10)

# Content frame
content_frame = tk.Frame(root, bg="#f5f5f5")
content_frame.pack(pady=10, fill="both", expand=True)

# Funny/Not Funny Checkboxes
funny_var = tk.BooleanVar()
not_funny_var = tk.BooleanVar()

checkbox_frame = tk.Frame(root, bg="#f5f5f5")
checkbox_frame.pack(pady=10)

funny_check = ttk.Checkbutton(checkbox_frame, text="Funny", variable=funny_var)
not_funny_check = ttk.Checkbutton(checkbox_frame, text="Not Funny", variable=not_funny_var)

funny_check.pack(side=tk.LEFT, padx=5)
not_funny_check.pack(side=tk.LEFT, padx=5)

# Label for slider
slider_label = tk.Label(root, text="How funny is it?", bg="#f5f5f5", font=("Arial", 12))
slider_label.pack()

# Slider for humor rating
slider_frame = tk.Frame(root, bg="#f5f5f5")
slider_frame.pack(pady=10, fill="x")

funny_slider = ttk.Scale(slider_frame, from_=1, to=5, orient=tk.HORIZONTAL)
funny_slider.pack(fill="x", padx=20)
funny_slider.set(3)  # Default to middle value

# Show numbers below slider
slider_ticks = tk.Frame(slider_frame, bg="#f5f5f5")
slider_ticks.pack(fill="x", padx=20)
for i in range(1, 6):
    label = tk.Label(slider_ticks, text=str(i), bg="#f5f5f5")
    label.pack(side="left", expand=True)

# Submit button
submit_btn = ttk.Button(root, text="Submit and Next", command=next_item)
submit_btn.pack(pady=20)

# Bind keyboard shortcuts
root.bind("f", funny_shortcut)
root.bind("n", not_funny_shortcut)
root.bind("<Return>", lambda e: next_item())

# Bind numpad keys to slider values
root.bind("1", lambda e: set_slider_value(1))
root.bind("2", lambda e: set_slider_value(2))
root.bind("3", lambda e: set_slider_value(3))
root.bind("4", lambda e: set_slider_value(4))
root.bind("5", lambda e: set_slider_value(5))

# Start the app
root.mainloop()
